# A5 小書建立器（Voila 版 v5 / 不用 FileUpload，一樣「選檔」）

這版**不使用 `ipywidgets.FileUpload`**，改以前端 `<input type="file" multiple>` 讀取檔案、轉成 Base64，**自動填入下方的圖片清單欄位**；  
Notebook 端只讀取這個欄位的文字（每行一張），因此能避開 MyBinder/行動裝置的相容性問題，同時保留「選檔」體驗。

- ✅ 仍支援貼 **圖片網址** 或 **Base64 資料 URI**（`data:image/...;base64,...`），一行一張。
- ✅ 下載用 `FileDownload`（正確 MIME、二進位輸出），且在下載前以 `python-docx` **自我驗證**。
- ✅ 支援 `[14pt]`、`[圖片]`、`[圖片60%]`、`[左圖80%][右圖50%]`、`[無圖]`。
- ✅ A5 頁面；封面 / 內頁 / 封底分節；封底不加頁碼、內頁可選頁碼。

**啟動：**
```bash
voila A5_Book_App_FIXED_v5.ipynb --show_tracebacks=True --debug
```


In [ ]:
# ===== 匯入 =====
import io, re, base64, urllib.request
import ipywidgets as widgets
from IPython.display import display, HTML

from docx import Document
from docx.shared import Mm, Pt, Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml import OxmlElement
from docx.oxml.ns import qn

from PIL import Image


In [ ]:
# ===== 工具與 DOCX 建構 =====

def ensure_rgb_jpeg(img_bytes, quality=90):
    im = Image.open(io.BytesIO(img_bytes))
    if im.mode not in ("RGB", "L"):
        im = im.convert("RGB")
    out = io.BytesIO()
    im.save(out, format="JPEG", quality=quality)
    return out.getvalue()

def parse_size_tag(text, default_pt=12):
    m = re.search(r"\[(\d+)pt\]\s*$", text)
    return int(m.group(1)) if m else default_pt

def strip_size_tag(text):
    return re.sub(r"\[\d+pt\]\s*$", "", text)

def has_skip_image_tag(text):
    return bool(re.search(r"\[無圖\]\s*$", text))

def strip_skip_image_tag(text):
    return re.sub(r"\[無圖\]\s*$", "", text)

def find_single_image_tag(text):
    m = re.search(r"\[(?:圖片)(\d+)?%?\]", text)
    if not m:
        return None
    pct = m.group(1)
    pct = int(pct) if pct else None
    return (pct, m.start(), m.end())

def find_double_image_tags(text):
    ml = re.search(r"\[左圖(\d+)?%?\]", text)
    mr = re.search(r"\[右圖(\d+)?%?\]", text)
    if not (ml and mr):
        return None
    lp = int(ml.group(1)) if ml.group(1) else None
    rp = int(mr.group(1)) if mr.group(1) else None
    return (lp, rp)

def set_section_to_a5(section):
    section.page_width = Mm(148)
    section.page_height = Mm(210)
    section.left_margin = Mm(15)
    section.right_margin = Mm(15)
    section.top_margin = Mm(15)
    section.bottom_margin = Mm(15)

def add_paragraph_with_size(doc, text, size_pt, align_center=False):
    p = doc.add_paragraph(text)
    p.alignment = WD_ALIGN_PARAGRAPH.CENTER if align_center else WD_ALIGN_PARAGRAPH.LEFT
    for run in p.runs:
        run.font.size = Pt(size_pt)
    return p

def add_single_image_paragraph(doc, img_bytes, width_pct=None):
    page_w_mm = 148 - 15 - 15
    page_w_in = page_w_mm / 25.4
    if width_pct is None:
        width_in = page_w_in * 0.9
    else:
        width_in = page_w_in * (width_pct / 100.0)
        width_in = max(1.0, min(page_w_in, width_in))
    safe = ensure_rgb_jpeg(img_bytes)
    doc.add_picture(io.BytesIO(safe), width=Inches(width_in))

def add_double_image_table(doc, left_bytes, right_bytes, l_pct, r_pct):
    page_w_mm = 148 - 15 - 15
    page_w_in = page_w_mm / 25.4
    base_in = page_w_in / 2.0
    l_in = base_in if l_pct is None else base_in * (l_pct / 100.0)
    r_in = base_in if r_pct is None else base_in * (r_pct / 100.0)
    l_in = max(1.0, min(page_w_in, l_in))
    r_in = max(1.0, min(page_w_in, r_in))

    table = doc.add_table(rows=1, cols=2)
    table.autofit = True
    cell = table.rows[0].cells[0]
    run = cell.paragraphs[0].add_run()
    run.add_picture(io.BytesIO(ensure_rgb_jpeg(left_bytes)), width=Inches(l_in))
    cell = table.rows[0].cells[1]
    run = cell.paragraphs[0].add_run()
    run.add_picture(io.BytesIO(ensure_rgb_jpeg(right_bytes)), width=Inches(r_in))

def add_page_number_footer(section, start_from=1):
    footer = section.footer
    p = footer.paragraphs[0] if footer.paragraphs else footer.add_paragraph()
    p.alignment = WD_ALIGN_PARAGRAPH.CENTER
    fld_simple = OxmlElement('w:fldSimple'); fld_simple.set(qn('w:instr'), 'PAGE')
    r = OxmlElement('w:r'); t = OxmlElement('w:t'); t.text = " "; r.append(t); fld_simple.append(r)
    p._p.append(fld_simple)
    sectPr = section._sectPr
    nodes = sectPr.xpath('./w:pgNumType')
    if nodes:
        pg = nodes[0]
    else:
        pg = OxmlElement('w:pgNumType'); sectPr.append(pg)
    pg.set(qn('w:start'), str(start_from))

def build_docx_bytes(cover_text, body_text, back_text, add_body_page_number, image_bytes_list):
    doc = Document()
    set_section_to_a5(doc.sections[0])
    add_paragraph_with_size(doc, cover_text.strip(), 20, align_center=True)

    doc.add_section()
    set_section_to_a5(doc.sections[-1])
    if add_body_page_number:
        add_page_number_footer(doc.sections[-1], start_from=1)

    paragraphs = [p.strip() for p in body_text.split('\n')]
    img_idx = 0

    for raw in paragraphs:
        if not raw:
            doc.add_paragraph("")
            continue
        size_pt = parse_size_tag(raw, default_pt=12)
        text = strip_size_tag(raw)
        skip_img = has_skip_image_tag(text)
        text = strip_skip_image_tag(text)

        dbl = find_double_image_tags(text)
        if dbl and (not skip_img) and (img_idx + 1 < len(image_bytes_list)):
            text_clean = re.sub(r"\[左圖\d*%?\]\s*\[右圖\d*%?\]", "", text).strip()
            if text_clean:
                add_paragraph_with_size(doc, text_clean, size_pt)
            l_pct, r_pct = dbl
            add_double_image_table(doc, image_bytes_list[img_idx], image_bytes_list[img_idx+1], l_pct, r_pct)
            img_idx += 2
            continue

        single = find_single_image_tag(text)
        if single and (not skip_img) and (img_idx < len(image_bytes_list)):
            pct, s, e = single
            text_clean = (text[:s] + text[e:]).strip()
            if text_clean:
                add_paragraph_with_size(doc, text_clean, size_pt)
            add_single_image_paragraph(doc, image_bytes_list[img_idx], width_pct=pct)
            img_idx += 1
            continue

        add_paragraph_with_size(doc, text, size_pt)

    doc.add_section()
    set_section_to_a5(doc.sections[-1])
    add_paragraph_with_size(doc, back_text.strip(), 12, align_center=True)

    buf = io.BytesIO(); doc.save(buf); data = buf.getvalue()
    _ = Document(io.BytesIO(data))  # 驗證
    return data

def load_image_from_line(line: str) -> bytes:
    line = line.strip()
    if not line:
        return None
    # data URL?
    if line.startswith('data:image'):
        head, b64 = line.split(',', 1)
        return base64.b64decode(b64)
    # 普通網址
    if line.startswith('http://') or line.startswith('https://'):
        with urllib.request.urlopen(line) as resp:
            return resp.read()
    # 純 base64（無 data: 前綴）
    try:
        return base64.b64decode(line, validate=True)
    except Exception:
        raise ValueError("無法辨識的圖片輸入：請提供圖片 URL、data:base64 或純 base64。")


In [ ]:
# ===== 使用者介面 =====

txt_cover = widgets.Textarea(
    value="我的 A5 小書\n（這是封面，會置中）",
    description='封面',
    layout=widgets.Layout(width='100%', height='80px')
)
txt_body = widgets.Textarea(
    value=(
        "這是一段內文。[14pt]\n"
        "這一段想放一張圖：[圖片]\n\n"
        "這一段想放雙圖：[左圖80%][右圖50%]\n"
        "若不想消耗圖片，即使段落含[圖片]，可在尾端加[無圖] 例如：\n"
        "這段含標記但跳過圖像[圖片][無圖]\n"
    ),
    description='內頁',
    layout=widgets.Layout(width='100%', height='180px')
)
txt_back = widgets.Textarea(
    value="封底文字（不加頁碼）",
    description='封底',
    layout=widgets.Layout(width='100%', height='60px')
)
chk_pagenum = widgets.Checkbox(value=True, description='內頁加頁碼')

images_input = widgets.Textarea(
    value="",
    description='圖片清單',
    placeholder="每行一張：可貼圖片 URL、data:image/...;base64,... 或純 base64",
    layout=widgets.Layout(width='100%', height='120px')
)

display(widgets.VBox([
    widgets.HBox([widgets.VBox([txt_cover, txt_back]), widgets.VBox([chk_pagenum])]),
    widgets.Label('—— 內頁 ——'),
    txt_body,
    widgets.Label('—— 圖片清單（每行一張） ——'),
    images_input
]))


In [ ]:
# ===== 前端「選檔→自動填入」的 HTML（不走 widgets.FileUpload） =====

model_id = images_input._model_id  # 讓前端 JS 取得此 Textarea 對應的 widget model

html = f"""<div style='margin:8px 0; padding:8px; border:1px dashed #aaa; border-radius:6px;'>
  <b>選擇圖片（多選）：</b>
  <input id="pick" type="file" accept="image/*" multiple />
  <div style="font-size:12px; color:#555; margin-top:4px;">
    會自動把選到的圖片轉成 <code>data:image/...;base64,...</code> 並填入上方的「圖片清單」欄位（每張一行）。
  </div>
</div>
<script>
(function() {{
  const input = document.getElementById('pick');
  if (!input) return;
  input.addEventListener('change', async (ev) => {{
    const files = Array.from(ev.target.files || []);
    if (!files.length) return;
    const toDataURL = (file) => new Promise((resolve, reject) => {{
      const fr = new FileReader();
      fr.onload = () => resolve(fr.result);
      fr.onerror = reject;
      fr.readAsDataURL(file);
    }});
    const lines = [];
    for (const f of files) {{
      try {{ lines.push(await toDataURL(f)); }}
      catch(e) {{ console.warn('read fail', e); }}
    }}
    const text = lines.join('\n');

    // 嘗試透過 widget manager 找到對應 Textarea 並寫入
    try {{
      const mgr = window.manager || window.widget_manager || (window.jupyter_widget_manager ?? null);
      if (mgr && mgr.get_model) {{
        const mdl = await mgr.get_model('{model_id}');
        mdl.set('value', text);
        mdl.save_changes();
        return;
      }}
    }} catch (e) {{
      console.warn('widget manager write failed', e);
    }}

    // 備援：若找不到 manager，就把內容塞進一個臨時 textarea，請使用者自己複製
    let fallback = document.getElementById('fallback_b64');
    if (!fallback) {{
      const box = document.createElement('div');
      box.innerHTML = '<p>複製以下內容到上方「圖片清單」：</p><textarea id="fallback_b64" style="width:100%;height:120px;"></textarea>';
      input.parentElement.appendChild(box);
      fallback = document.getElementById('fallback_b64');
    }}
    fallback.value = text;
    alert('已產生 Base64，若上方欄位沒有自動出現，請複製備援框內容貼上。');
  }});
}})();
</script>
"""

display(HTML(html))


In [ ]:
# ===== 下載按鈕 =====

def make_docx_bytes_for_download():
    # 將圖片清單的每行轉成 bytes
    lines = [ln.strip() for ln in images_input.value.splitlines() if ln.strip()]
    imgs = []
    for ln in lines:
        imgs.append(load_image_from_line(ln))
    data = build_docx_bytes(
        cover_text=txt_cover.value,
        body_text=txt_body.value,
        back_text=txt_back.value,
        add_body_page_number=chk_pagenum.value,
        image_bytes_list=imgs
    )
    _ = Document(io.BytesIO(data))  # 驗證
    return data

widgets.FileDownload(
    data=make_docx_bytes_for_download,
    filename='A5_book_fixed_v5.docx',
    description='⬇️ 下載 DOCX（已驗證）',
    mime='application/vnd.openxmlformats-officedocument.wordprocessingml.document',
)
